In [1]:
import warnings, os
warnings.filterwarnings("ignore")

from copy import copy
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import roc_auc_score, matthews_corrcoef, precision_score, recall_score, f1_score


import seaborn as sns
import matplotlib.pyplot as plt
from joblib import dump, load

from aggmap import AggMap, AggModel, loadmap
from aggmap.AggModel import load_model, save_model
from aggmap import show


np.random.seed(666) #just for reaptable results


def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_pred

    '''
    the metrics are taken from orignal paper:
    Meta-Signer: Metagenomic Signature Identifier based on Rank Aggregation of Features
    https://github.com/YDaiLab/Meta-Signer/blob/bd6a1cd98d1035f848ecb6e53d9ee67a85871db2/src/utils/metasigner_io.py#L34
    '''
    auc = roc_auc_score(y_true, y_score, average='weighted')        
    mcc = matthews_corrcoef(y_true, y_pred)
    pres = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print('roc-auc: %.3f, mcc: %.3f, pres: %.3f, recall: %.3f, f1: %.3f' % (auc, mcc, pres, recall, f1))

    return auc, mcc, pres, recall, f1

# read data

In [2]:
task = 'T2D'
data_path = '../01_data/species_level/%s/' % (task)
save_dir = '%s_results' % task
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

dfa = pd.read_csv(os.path.join(data_path, 'abundance.tsv'),sep='\t', header=None, index_col=0)
dfy = pd.read_csv(os.path.join(data_path, 'labels.txt'),sep='\t', header=None)
dfx = dfa.T
dfy = pd.get_dummies(dfy[0].map({'t2d':1, 'n':0}))
Y = dfy.values

# 10FCV

In [ ]:
gpuid = 7

outer_fold = 10
repeat_seeds = [8, 16, 32, 64, 128, 256, 1024, 2048, 4096, 8192] #10 repeats random seeds 8, 16, 32, 64, 128

each_fold_results = []
run_all_res = []

for i, repeat_seed in enumerate(repeat_seeds): 
    outer = StratifiedKFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(range(len(dfy)), dfy.idxmax(axis=1))
    run_one_res = []
    for j, idx in enumerate(outer_idx):
        fold_num = "fold_%s" % str(j).zfill(2) 
        print('#'*50 + ' repeat_seed: %s; %s ' % (repeat_seed, fold_num) + '#'*50 )
        
        train_idx, test_idx = idx
        dfx_train = dfx.iloc[train_idx]
        dfy_train = dfy.iloc[train_idx]
        
        ## get best parameters 
        if (i == 0) & (j == 0):
            from tune import finetune_HPs
            best_fill, best_scale_method, best_channel_number, best_epochs, batch_size = finetune_HPs(dfx_train, dfy_train, gpuid=gpuid)
            featHPs = {"best_fill":best_fill, "best_scale_method":best_scale_method, "best_channel_number":best_channel_number}
            dfx = np.log(dfx + best_fill)
            mp = AggMap(dfx, metric = 'correlation')
            mp = mp.fit(cluster_channels = best_channel_number, verbose = 0, var_thr = 0)
            mp.plot_grid(save_dir)
            mp.plot_scatter(save_dir)
            mp.save(os.path.join(save_dir, 'agg.mp'))
            X = mp.batch_transform(dfx.values, scale = best_scale_method) # NaN values should be the lowest value
            
        testY = Y[test_idx]
        testX = X[test_idx]
        
        trainX = X[train_idx]
        trainY = Y[train_idx]

        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))

        clf = AggModel.MultiClassEstimator(epochs = best_epochs,  batch_size = batch_size, verbose = 0, gpuid=gpuid) #
        clf.fit(trainX, trainY)  #, 
        
        ## save model for explaination
        if i == 0:
            clf.save_model(os.path.join(save_dir, '%s.model' % fold_num))
            paras = clf.get_params()
            paras.update({'featHPs':featHPs})
            pd.Series(paras).to_json(os.path.join(save_dir, 'HPs.json'))

        pred_proba = clf.predict_proba(testX)
        y_true = testY[:,1] 
        y_score = pred_proba[:,1]
        y_pred = np.argmax(pred_proba, axis=1)
        
        dfr = pd.DataFrame([y_true, y_score, y_pred]).T
        dfr.columns = ['y_true', 'y_score', 'y_pred']
        dfr.index = dfy.iloc[test_idx].index
        auc, mcc, pres, recall, f1  = score(dfr)
        run_one_res.append(dfr)
        ts = pd.Series([auc, mcc, pres, recall, f1, i, repeat_seed]).round(3)
        ts.index = ['auc', 'mcc', 'pres', 'recall', 'f1', 'i', 'repeat_seed']
        print(ts.to_dict())
        each_fold_results.append(ts.to_dict())
    run_all_res.append(pd.concat(run_one_res))

################################################## repeat_seed: 8; fold_00 ##################################################
2021-08-17 17:46:59,376 - INFO - [bidd-aggmap] - Calculating distance ...
2021-08-17 17:46:59,397 - INFO - [bidd-aggmap] - the number of process is 16


100%|##########| 606/606 [00:00<00:00, 791.80it/s]


2021-08-17 17:47:06,719 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2021-08-17 17:47:11,260 - INFO - [bidd-aggmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)


  0%|          | 0/396 [00:00<?, ?it/s]

2021-08-17 17:47:11,710 - INFO - [bidd-aggmap] - Finished


...

100%|##########| 396/396 [00:00<00:00, 868.83it/s]


{'best_loss': 0.575, 'best_epoch': 10, 'scale_method': 'minmax', 'fold_num': 'fold_01'}
{'best_loss': 0.652, 'best_epoch': 6, 'scale_method': 'minmax', 'fold_num': 'fold_03'}
{'best_loss': 0.609, 'best_epoch': 8, 'scale_method': 'minmax', 'fold_num': 'fold_05'}
{'best_loss': 0.687, 'best_epoch': 2, 'scale_method': 'minmax', 'fold_num': 'fold_07'}


 12%|#2        | 48/396 [00:00<00:00, 437.32it/s]

{'best_loss': 0.683, 'best_epoch': 2, 'scale_method': 'minmax', 'fold_num': 'fold_09'}


 35%|###5      | 140/396 [00:00<00:00, 505.78it/s]

In [10]:
pd.DataFrame(each_fold_results).groupby('repeat_seed').mean().mean()

auc       0.74492
mcc       0.38316
pres      0.69359
recall    0.69004
f1        0.68833
i         4.50000
dtype: float64

In [11]:
pd.DataFrame(each_fold_results).groupby('repeat_seed').std().mean()

auc       0.074003
mcc       0.155460
pres      0.077954
recall    0.077341
f1        0.077991
i         0.000000
dtype: float64

In [12]:
pd.DataFrame(each_fold_results).to_csv(os.path.join(save_dir, 'performance_results.csv'))

# feature importance

In [ ]:
all_imps = []
for i in range(10):
    clf = load_model(os.path.join(save_dir, 'fold_%s.model' % str(i).zfill(2)))
    dfe = clf.explain_model(mp, clf.X_, clf.y_, binary_task=True, apply_logrithm=False)
    df_imp = dfe.col_1_importance.to_frame(name = 'fold_%s_imp' % str(i).zfill(2))
    all_imps.append(df_imp)

  1%|1         | 8/625 [00:00<00:08, 70.78it/s]

calculating feature importance for column 1 ...


  1%|1         | 7/625 [00:00<00:09, 62.91it/s]

calculating feature importance for column 1 ...


  1%|1         | 8/625 [00:00<00:07, 78.82it/s]

calculating feature importance for column 1 ...


  1%|1         | 7/625 [00:00<00:10, 61.35it/s]

calculating feature importance for column 1 ...


 51%|#####     | 317/625 [00:27<00:23, 13.26it/s]

In [ ]:
dfi = dfe[dfe.columns[:-1]]
dfi['scatter_x'] = dfi.v.map(mp.df_embedding.x)
dfi['scatter_y'] = dfi.v.map(mp.df_embedding.y)

dfimp_all = pd.concat(all_imps, axis=1)
dfi = dfi.join(dfimp_all.mean(axis=1).to_frame(name = 'avg_imp'))
dfi = dfi.join(dfimp_all)
dfi.to_csv(os.path.join(save_dir, 'feature_imp_results.csv'))

In [ ]:
dfi